[View in Colaboratory](https://colab.research.google.com/github/upadhyayprakash/EIP/blob/master/PRAKASH_BATCH_3_ASSIGNMENT4B_NEW.ipynb)

**Attempt-2: Assigment 4B of EIP Batch 3 - Prakash Upadhyay**</br>
**Task: ** Beat the Validation Score of **76.09%** accuracy.

- Acquired Accuracy: **84.17%**

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU # Added by Prakash
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 40
l = 40
num_filter = 32
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# Added by Prakash
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    #AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    #flat = Flatten()(AvgPooling)
    flat = Flatten()(relu)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# START: Added Pooling & Dropout by Prakash

Second_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(First_Conv2D)
DropOut_Conv2D = Dropout(0.2)(Second_Conv2D)
AvgPooling2D_Conv2D = AveragePooling2D(pool_size=(2,2))(DropOut_Conv2D)

# END: Added Pooling & Dropout by Prakash


# START: Added Pooling & Dropout by Prakash
#BatchNorm = BatchNormalization()(input)
#relu = Activation('relu')(BatchNorm)
    
#Second_Conv2D = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)

#DropOut_Conv2D = Dropout(0.2)(Second_Conv2D)
#AvgPooling2D_Conv2D = AveragePooling2D(pool_size=(2,2))(DropOut_Conv2D)

# END: Added Pooling & Dropout by Prakash

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
#Last_Block = add_denseblock(Third_Block,  num_filter, dropout_rate)

output = output_layer(Third_Block)

In [50]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 32, 32, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_118 (BatchN (None, 32, 32, 32)   128         conv2d_121[0][0]                 
__________________________________________________________________________________________________
activation_118 (Activation)     (None, 32, 32, 32)   0           batch_normalization_118[0][0]    
__________________________________________________________________________________________________
conv2d_123

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [52]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 288s 6ms/step - loss: 1.6660 - acc: 0.4429 - val_loss: 2.0007 - val_acc: 0.4004
Epoch 2/40
19200/50000 [==========>...................] - ETA: 2:37 - loss: 1.2891 - acc: 0.5742

50000/50000 [==============================] - 275s 6ms/step - loss: 1.2555 - acc: 0.5869 - val_loss: 1.2960 - val_acc: 0.5723
Epoch 3/40
45568/50000 [==========================>...] - ETA: 22s - loss: 1.0516 - acc: 0.6539

50000/50000 [==============================] - 275s 6ms/step - loss: 1.0467 - acc: 0.6549 - val_loss: 1.4772 - val_acc: 0.6026
Epoch 4/40
50000/50000 [==============================] - 276s 6ms/step - loss: 0.9111 - acc: 0.6991 - val_loss: 0.9236 - val_acc: 0.6914
Epoch 5/40
 3328/50000 [>.............................] - ETA: 3:58 - loss: 0.8319 - acc: 0.7323

50000/50000 [==============================] - 275s 6ms/step - loss: 0.7954 - acc: 0.7341 - val_loss: 0.8287 - val_acc: 0.7294
Epoch 6/40
39424/50000 [======================>.......] - ETA: 54s - loss: 0.6845 - acc: 0.7648

50000/50000 [==============================] - 276s 6ms/step - loss: 0.6816 - acc: 0.7656 - val_loss: 0.8410 - val_acc: 0.7190
Epoch 7/40
50000/50000 [==============================] - 276s 6ms/step - loss: 0.6158 - acc: 0.7852 - val_loss: 0.7972 - val_acc: 0.7414
Epoch 8/40
 1792/50000 [>.............................] - ETA: 4:08 - loss: 0.5363 - acc: 0.8047

...

50000/50000 [==============================] - 275s 6ms/step - loss: 0.1194 - acc: 0.9569 - val_loss: 0.9411 - val_acc: 0.7956
Epoch 34/40
50000/50000 [==============================] - 276s 6ms/step - loss: 0.1160 - acc: 0.9589 - val_loss: 0.8433 - val_acc: 0.8231
Epoch 35/40
 1536/50000 [..............................] - ETA: 4:08 - loss: 0.1023 - acc: 0.9642

50000/50000 [==============================] - 276s 6ms/step - loss: 0.1146 - acc: 0.9572 - val_loss: 0.9205 - val_acc: 0.8120
Epoch 36/40
38528/50000 [======================>.......] - ETA: 59s - loss: 0.1115 - acc: 0.9602

50000/50000 [==============================] - 276s 6ms/step - loss: 0.1130 - acc: 0.9593 - val_loss: 0.7807 - val_acc: 0.8417
Epoch 37/40
50000/50000 [==============================] - 276s 6ms/step - loss: 0.1070 - acc: 0.9616 - val_loss: 0.8331 - val_acc: 0.8295
Epoch 38/40
 1536/50000 [..............................] - ETA: 4:08 - loss: 0.0847 - acc: 0.9674

50000/50000 [==============================] - 275s 5ms/step - loss: 0.1059 - acc: 0.9615 - val_loss: 0.7733 - val_acc: 0.8335
Epoch 39/40
38528/50000 [======================>.......] - ETA: 58s - loss: 0.1002 - acc: 0.9644

50000/50000 [==============================] - 275s 6ms/step - loss: 0.1031 - acc: 0.9635 - val_loss: 0.9342 - val_acc: 0.8153
Epoch 40/40
50000/50000 [==============================] - 275s 6ms/step - loss: 0.0966 - acc: 0.9656 - val_loss: 0.7715 - val_acc: 0.8381


In [53]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 25s 3ms/step
Test loss: 0.7715331559419631
Test accuracy: 0.8381
